In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import numpy as np
max_long = 25

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import sys
sys.path.append('/content/gdrive/My Drive')

In [5]:
!pip install unidecode

In [6]:
import data
data.get_data, data.clean_data

(<function data.get_data>, <function data.clean_data>)

In [7]:
import token_glove
token_glove.create_list, token_glove.token_ize, token_glove.voc_token, token_glove.dict_token, token_glove.token_tweet, token_glove.glove_vector

(<function token_glove.create_list>,
 <function token_glove.token_ize>,
 <function token_glove.voc_token>,
 <function token_glove.dict_token>,
 <function token_glove.token_tweet>,
 <function token_glove.glove_vector>)

In [8]:
import metrics
metrics.recall_m, metrics.precision_m, metrics.f1_m

(<function metrics.recall_m>,
 <function metrics.precision_m>,
 <function metrics.f1_m>)

In [9]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [10]:
df = data.get_data()
cleaned_df = data.clean_data(df)

In [11]:
text = token_glove.create_list(cleaned_df['text'])
token = token_glove.token_ize(text)
X,token_izer = token_glove.token_tweet(text, token)

In [12]:
word_vector_matrix = token_glove.glove_vector(token)

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Activation, Dropout, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
vocab_size = token_glove.voc_token(token)
vec_size = 200

In [15]:
y = cleaned_df['target']

In [16]:
model = Sequential()

es = EarlyStopping(patience=5,restore_best_weights= True)

model.add(Embedding(vocab_size, vec_size, input_length=max_long, weights = [word_vector_matrix], trainable = True))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(64, recurrent_dropout=0.5,))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate = 0.0001),metrics=['accuracy'])
history = model.fit(X, y, batch_size=16, epochs = 400, validation_split=0.1, callbacks = [es])

Epoch 1/400
429/429 [==============================] - 18s 34ms/step - loss: 0.5636 - accuracy: 0.7202 - val_loss: 0.4229 - val_accuracy: 0.8110
Epoch 2/400
429/429 [==============================] - 14s 33ms/step - loss: 0.4390 - accuracy: 0.8072 - val_loss: 0.4090 - val_accuracy: 0.8255
Epoch 3/400
429/429 [==============================] - 14s 33ms/step - loss: 0.4004 - accuracy: 0.8302 - val_loss: 0.4080 - val_accuracy: 0.8241
Epoch 4/400
429/429 [==============================] - 14s 33ms/step - loss: 0.3703 - accuracy: 0.8467 - val_loss: 0.4154 - val_accuracy: 0.8202
Epoch 5/400
429/429 [==============================] - 14s 33ms/step - loss: 0.3388 - accuracy: 0.8641 - val_loss: 0.4261 - val_accuracy: 0.8163
Epoch 6/400
429/429 [==============================] - 14s 33ms/step - loss: 0.3055 - accuracy: 0.8822 - val_loss: 0.4505 - val_accuracy: 0.8150
Epoch 7/400
429/429 [==============================] - 14s 33ms/step - loss: 0.2754 - accuracy: 0.8980 - val_loss: 0.4778 - val_ac

#Phase de Test

In [17]:
df = data.get_data(test=True)
test_df = data.clean_data(df)

In [28]:
test_df.shape

(3263, 2)

In [29]:
text_test = token_glove.create_list(test_df['text'])
encoded_text_test = token_izer.texts_to_sequences(text_test)
X_test = pad_sequences(encoded_text_test, maxlen=max_long, padding='post')

In [30]:
results = model.predict(X_test)

In [31]:
results

array([[0.88107675],
       [0.8874686 ],
       [0.85881037],
       ...,
       [0.86547685],
       [0.80852526],
       [0.46564394]], dtype=float32)

In [21]:
def arrondi(n):
  if n >= 0.5:
    return 1
  else :
    return 0

In [32]:
to_submit = [arrondi(x[0]) for x in results]

In [33]:
to_submit_df = pd.DataFrame(to_submit)

In [34]:
to_submit_df.rename(columns={0: 'target'}, inplace=True)

In [35]:
to_submit_df['id'] = test_df.id

In [36]:
to_submit_df = to_submit_df.set_index('id')

In [37]:
to_submit_df['target'].value_counts()

0    2103
1    1160
Name: target, dtype: int64

In [38]:
to_submit_df.to_csv('/content/gdrive/My Drive/submission7.csv')

In [39]:
model.save('/content/gdrive/My Drive/RNN_camille_version_finale')

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/RNN_camille_version_finale/assets
